# (Un)supervised node prediction

##  Random Walk based embeddings

Use data from https://github.com/shchur/gnn-benchmark/tree/master/data/npz , particulary use Amazon Computers dataset, see description in a corresponding paper https://arxiv.org/pdf/1811.05868.pdf. 
> Recall that this network has small amount of isolated nodes.

1. Run `DeepWalk` to get embeddings of size 32.
2. Using `kmeans` with appropriate number of clusters (somewhere between 6 and 12) compute node labels. Compare the result with ground truth communities using `adjusted_rand_index`. Compute corresponding `Modularities`.
3. Run your favourite dimensionality reduction algorithm to get a 2 dimensional embedding.
4. Compare results (repeate 1-3) with embeddings of size 64 and 128.
5. Compare results with supervised Logistic Regression on data feature matrix (without using network data).


Recommended for visualization:

- Dmitry Ulyanov has nice multicore tsne implementation https://github.com/DmitryUlyanov/Multicore-TSNE
- Recent paper from Aleksandr Artemenkov and Maxim Panov https://arxiv.org/pdf/2001.11411.pdf with implementation https://github.com/alartum/ncvis . Reported to be superior to TSNE for the purpose of 2 dimensional visualization.

In [188]:
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [4]:
!deepwalk --help

usage: deepwalk [-h] [--debug] [--format FORMAT] --input [INPUT] [-l LOG]
                [--matfile-variable-name MATFILE_VARIABLE_NAME]
                [--max-memory-data-size MAX_MEMORY_DATA_SIZE]
                [--number-walks NUMBER_WALKS] --output OUTPUT
                [--representation-size REPRESENTATION_SIZE] [--seed SEED]
                [--undirected UNDIRECTED] [--vertex-freq-degree]
                [--walk-length WALK_LENGTH] [--window-size WINDOW_SIZE]
                [--workers WORKERS]

options:
  -h, --help            show this help message and exit
  --debug               drop a debugger if an exception is raised. (default:
                        False)
  --format FORMAT       File format of input file (default: adjlist)
  --input [INPUT]       Input graph file (default: None)
  -l LOG, --log LOG     log verbosity level (default: INFO)
  --matfile-variable-name MATFILE_VARIABLE_NAME
                        variable name of adjacency matrix inside a .mat file.
     

In [119]:
def load_npz_to_sparse_graph(file_name):
    """Load a SparseGraph from a Numpy binary file.
    from https://github.com/shchur/gnn-benchmark/blob/master/gnnbench/data/io.py
    """
    
    from scipy.sparse import csr_matrix
    
    with np.load(file_name) as loader:
        loader = dict(loader)
        adj_matrix = csr_matrix((loader['adj_data'], loader['adj_indices'], loader['adj_indptr']),
                                   shape=loader['adj_shape'])
        attr_matrix = csr_matrix((loader['attr_data'], loader['attr_indices'], loader['attr_indptr']),
                                        shape=loader['attr_shape'])
        labels = loader['labels']
        class_names = loader.get('class_names')

    return adj_matrix, attr_matrix, labels, class_names

def pca_reduction(X, n_components=2):
    pca = PCA(n_components=n_components)
    
    return pca.fit_transform(X)

def kmeans_on_pca_emdedding(embedding_file, y, k=10, pca_n_components=2):
    # load resulting embedding

    with open(embedding_file, 'r') as f:
        # first line is a header containing `number of nodes` and `embeddings size`
        n, m = map(int, f.readline().split())
        n = 13752
        node_embeddings = np.zeros((n, m))
        # remaining lines are node_id and embedding vector components
        for line in f.readlines():  
            try:
                node_id, *emb = line.split(' ')
                node_embeddings[int(node_id)] = list(map(float, emb))
            except:
                pass
            
    # pca
    node_embeddings_pca = pca_reduction(node_embeddings, n_components=pca_n_components)
    
    # k-means      
    node_colors_kmeans = KMeans(k).fit_predict(node_embeddings_pca)
    order = np.argsort(node_colors_kmeans)
    rand_score = adjusted_rand_score(y, node_colors_kmeans)

    return node_colors_kmeans, order, rand_score

def kmeans_on_embedding(embedding_file, y, k=10):
    # load resulting embedding

    with open(embedding_file, 'r') as f:
        # first line is a header containing `number of nodes` and `embeddings size`
        n, m = map(int, f.readline().split())
        n = 13752
        node_embeddings = np.zeros((n, m))
        # remaining lines are node_id and embedding vector components
        for line in f.readlines():  
            try:
                node_id, *emb = line.split(' ')
                node_embeddings[int(node_id)] = list(map(float, emb))
            except:
                pass
            
    # k-means      
    node_colors_kmeans = KMeans(k).fit_predict(node_embeddings)
    order = np.argsort(node_colors_kmeans)
    rand_score = adjusted_rand_score(y, node_colors_kmeans)

    return node_colors_kmeans, order, rand_score

In [91]:
graph_file = 'amazon_electronics_computers.npz'
edgelist_file = 'sparce_graph.edgelist'
embedding_file32 = 'sparce_graph.embedding32'
embedding_file64 = 'sparce_graph.embedding64'
embedding_file128 = 'sparce_graph.embedding128'

In [177]:
adj, X, y, class_names = load_npz_to_sparse_graph(graph_file)
print(adj.todense().shape)
sparse_g = nx.from_numpy_array(adj)

(13752, 13752)


In [71]:
with open(edgelist_file, 'w') as f:
    for edge in sparse_g.edges:
        f.write(f'{edge[0]} {edge[1]}\n')

### embedding 32

In [93]:
!deepwalk --seed 41 --representation-size 32 --format edgelist --input sparce_graph.edgelist --output sparce_graph.embedding32

Number of nodes: 13471
Number of walks: 134710
Data size (walks*length): 5388400
Walking...
Training...
2023-11-09 17:08:45 WARNING word2vec.py: 1545 Both hierarchical softmax and negative sampling are activated. This is probably a mistake. You should set either 'hs=0' or 'negative=0' to disable one of them. 


In [170]:
node_colors_kmeans32, order32, rand_score32 = kmeans_on_embedding(
                                                                    embedding_file32, 
                                                                    y, 
                                                                    k=10
                                                                  )

node_colors_kmeans32_pca, order32_pca, rand_score32_pca = kmeans_on_pca_emdedding(
                                                                                    embedding_file32, 
                                                                                    y, 
                                                                                    k=10
                                                                                  )

print('adjusted rand score on 32 embedding:', np.around(rand_score32, 3))
print('adjusted rand score on 32 embedding with pca reduction:', np.around(rand_score32_pca, 3))

adjusted rand score on 32 embedding: 0.409
adjusted rand score on 32 embedding with pca reduction: 0.405


### embedding 64

In [122]:
!deepwalk --seed 41 --representation-size 64 --format edgelist --input sparce_graph.edgelist --output sparce_graph.embedding64

Number of nodes: 13471
Number of walks: 134710
Data size (walks*length): 5388400
Walking...
Training...
2023-11-09 17:21:29 WARNING word2vec.py: 1545 Both hierarchical softmax and negative sampling are activated. This is probably a mistake. You should set either 'hs=0' or 'negative=0' to disable one of them. 


In [169]:
node_colors_kmeans64, order64, rand_score64 = kmeans_on_embedding(
                                                                    embedding_file64, 
                                                                    y, 
                                                                    k=10
                                                                  )

node_colors_kmeans64_pca, order64_pca, rand_score64_pca = kmeans_on_pca_emdedding(
                                                                                    embedding_file64, 
                                                                                    y, 
                                                                                    k=10
                                                                                  )

print('adjusted rand score on 64 embedding:', np.around(rand_score64, 3))
print('adjusted rand score on 64 embedding with pca reduction:', np.around(rand_score64_pca, 3))

adjusted rand score on 64 embedding: 0.429
adjusted rand score on 64 embedding with pca reduction: 0.382


### embedding 128

In [139]:
!deepwalk --seed 41 --representation-size 128 --format edgelist --input sparce_graph.edgelist --output sparce_graph.embedding128

Number of nodes: 13471
Number of walks: 134710
Data size (walks*length): 5388400
Walking...
Training...
2023-11-09 17:25:44 WARNING word2vec.py: 1545 Both hierarchical softmax and negative sampling are activated. This is probably a mistake. You should set either 'hs=0' or 'negative=0' to disable one of them. 


In [158]:
node_colors_kmeans128, order128, rand_score128 = kmeans_on_embedding(
                                                                      embedding_file128, 
                                                                      y, 
                                                                      k=10
                                                                    )

node_colors_kmeans128_pca, order128_pca, rand_score128_pca = kmeans_on_pca_emdedding(
                                                                                      embedding_file128, 
                                                                                      y, 
                                                                                      k=10
                                                                                    )

print('adjusted rand score on 128 embedding:', np.around(rand_score128, 3))
print('adjusted rand score on 128 embedding with pca reduction:', np.around(rand_score128_pca, 3))

adjusted rand score on 128 embedding: 0.429
adjusted rand score on 128 embedding with pca reduction: 0.39


### Logistic Regression

In [193]:
X_nparr = np.asarray(X.todense())
n = int(len(y)*0.15) # разделим выборку на train и test, 85% и 15% соотвественно
N = len(y) - n

clf = LogisticRegression(random_state=0).fit(X_nparr[:N], y[:N])
print('score:', np.around(clf.score(X_nparr[N:], y[N:]), 3))
logreg_labels = clf.predict(X_nparr[N:])
print('adjusted rand score on logreg labels:', np.around(adjusted_rand_score(y[N:], logreg_labels), 3))

score: 0.828
adjusted rand score on logreg labels: 0.647


### результаты

#### Сравнительная таблица

| метод | adjusted rand score | 
| --- | --- |
| deep walk emb 32 | 0.409 | 
| deep walk emb 32 -> 2 | 0.405 | 
| deep walk emb 64 | 0.428 | 
| deep walk emb 64 -> 2 | 0.382| 
| deep walk emb 128 | 0.429 | 
| deep walk emb 128 -> 2 |0.39 | 
| log reg | 0.647 | 

### Вывод

анализируя результаты классификации по эмбеддингам deep walk разной размерности, наилучший результат дает кластеризация на эмбеддингах размерности 128, однако результат на эмбеддингах размерности 64 не намного отсает от него. 

в качестве алгоритма понижения размерности эмбеддингов до размерности 2 использовался PCA. на сжатых эмбеддингах лучший результат получился для 32 -> 2 врианта. Видимо, потому сжимая эмбеддинги большей размерности в 2 не удается сохранить достаточно нужной информации, чтобы сохранить качество кластеризации. 

наилучший результат классификации дает логистическая регрессия с adjusted rand score 0.647.